# Tearing the Mask off Python Decorators
## Deep look into some advanced concepts
<img src='images/jelly.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@infonautica?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Leonid Danilov</a>
        on 
        <a href='https://www.pexels.com/photo/photo-of-jellyfish-2690765/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Introduction

### Functions Are Objects

One of the many things you will love about Python is its ability to represent anything as objects and functions are no exception. For people who is first reading this, passing a function as an argument to another function may seem strange but is completely legal to do so:

In [1]:
def my_func(arg=42):
    """
    Placeholder function
    """
    print('Printing the function\'s argument')

print(my_func)

<function my_func at 0x000001457A428550>


> Important note to be useful in later sections: Using the function with parentheses, `my_func()`, is called __'calling'__ the function while writing without, `my_func`, is called __referencing__. As you have seen, `print(my_func)` prints the function's
index in memory.

As objects go, functions are absolutely the same as:
- str
- int, float
- pandas.DataFrame
- list, tuple, dict
- modules: os, datatime, numpy

You may assign functions to a new variable and use it to call the function:

In [2]:
new_func = my_func
new_func()

Printing the function's argument


Now this variable also contains the function's attributes:

In [3]:
# Get the docstring of a function
new_func.__doc__

'\n    Placeholder function\n    '

In [4]:
# Get function name
new_func.__name__

'my_func'

In [5]:
# Default argument values
new_func.__defaults__

(42,)

You can also store each function in other objects such as lists, dictionaries and call them:

In [6]:
funcs = [str.lower, print, range, str.startswith]
for func in funcs:
    print(f'The function name is \'{func.__name__}\'')

The function name is 'lower'
The function name is 'print'
The function name is 'range'
The function name is 'startswith'


In [7]:
func_dict = {
    'lower': str.lower,
    'print': print,
    'range': range,
    'startswith': str.startswith
}

func_dict['print'](func_dict['lower']('PYTHON'))

python


### Scope

Consider this conversation between Bob and Job:
- Bob: 'Jon, why did not you come to the lesson yesterday?'
- Jon: 'I had a flu...'

Not the best of stories but when Bob asks the reason of Jon's absence in yesterday's class, we know he is referring to the Jon standing next to him not some random Jon in another country. As humans, it is not difficult to notice this but programming languages use something called scope to tell which name we are referring to in our programs.

In Python, names can be variables, function names, module names, etc. 

Consider these two variables:

In [8]:
a = 24
b = 42
print(a)

24


Here, `print` had no trouble to tell that we are referring to the `a` we just defined. Now consider this:

In [9]:
def foo():
    a = 100
    print(a)

What do you think will happen if we run `foo`? Will it print 24 or 100?

In [10]:
foo()

100


How did Python differentiate between the `a` we defined in the beginning or in the function? This is where scope gets interesting, because we are introducing layers of scope:

<img src='images/1.png'></img>

The above image shows the scope for this little script:

In [11]:
a = 24
b = 42
print(a)

def foo():
    a = 100
    print(a)
    
foo()

24
100


The global scope is the overall scope of your script/program. Variables, functions, etc. with the same indentation level as `a` and `b` in the beginning will be in the global scope. For example, `foo` is in the global scope but its variable `a` is in the scope that is local to `foo`. 

In one global scope, there can be many local scopes. For example, each temporary variables in `for` loops and list comprehensions, return values of context managers will be local inside their code block and cannot be accessed from the global scope.

Here, we add a for loop at the end of our little script:

In [12]:
for num in range(10):
    print(num)  # num is local to this for loop

0
1
2
3
4
5
6
7
8
9


So, a rule of thumb is that Python interpreter will not be able to access a name that is in one level inner of the current scope. 

There is also a bigger level of scope outside `global`:

<img src='images/2.png'></img>

Built-in scope contains all the modules and packages you installed with `Python`, `pip` or `conda`.

Now, let's explore another case. In our `foo` function, we want to modify the value of global `a`. We want it to be a string but if we write `a = 'some text'` inside the function, Python will just create a new variable without modify the global `a`. 

Python provides us with a keyword that lets us specify we are referring to names in the `global` scope:

In [13]:
# Before
def foo():
    a = 'some text'
    print(a)

foo()
print(a)

some text
24


In [14]:
# Using `global` keyword
# After
def foo():
    global a
    a = 'some text'
    print(a)
foo()
print(a)

some text
some text


Writing `global <name>` will let us modify the values of names in the `global` scope. 

BTW, bad news, I left out one level of scope in the above image. Between `global` and `local`, there is one level we did not cover:

<img src='images/3.png'></img>

`nonlocal` scope comes in to the play when we have, for example, nested functions:

In [15]:
def outer():
    # Create a dummy variable
    my_var = 'Python'
    
    def inner():
        # Try to change the value of the dummy
        my_var = 'Data Science'
        print(my_var)
    # Call the inner function which tries to modify `my_var`
    inner()
    # Check if successful
    print(my_var)

outer()

Data Science
Python


In nested function `outer`, we first create a variable called `my_var` and assign it to the string `Python`. Then we decide to create a new `inner` function and want to assign `my_var` a new value, `Data Science` and print it. But if we run it, we see that `my_var` is still assigned to 'Python'. We cannot use `global` keyword since `my_var` is not in the global scope. 

For such cases, you can use `nonlocal` keyword which gives access to all the names in the scope of the outer function (nonlocal) but not the `global`:

In [16]:
def outer():
    # Create a dummy variable
    my_var = 'Python'
    
    def inner():
        # Try to change the value of the dummy with nonlocal
        nonlocal my_var
        my_var = 'Data Science'
        print(my_var)
    # Call the inner function which tries to modify `my_var`
    inner()
    # Check if successful
    print(my_var)

outer()

Data Science
Data Science


In conclusion, scope tells the Python interpreter where to look for names in our program. There can be four levels of scope in a single script/program:
- Built-in: all the package names installed with Python, `pip` and `conda`
- Global: general scope, all names that has no indentation in the script
- Local: contains local variables in each code block such as functions, loops, list comprehensions, etc.
- Nonlocal: an extra level of scope between `global` and `local` in the case of nested functions

### Closures

Before I explain how decorators work, we need to talk about closures too. Let's start with an example:

In [17]:
def foo():
    x = 42
    
    def bar():
        print(x)
    
    return bar

We create a nested function `bar` inside `foo` and return it. `bar` tries to print the value `of` x:

In [18]:
var = foo()  # var is assigned to `bar` because we are retruning it inside `foo`

When we write `var = foo()`, we are assigning the `bar` function to `var`. Now `var` can be used to call `bar`. When we call it, it prints out 42. 

In [19]:
var()

42


But wait a minute, how does `var` know anything about `x`? `x` is defined in `foo`'s scope not `bar`'s. You would think that `x` would be accessible outside the scope of `foo`. That's where closures come in.

> Closure is a built-in memory of a function that contains all the nonlocal names (in a tuple) the function needs to run!

So, when `foo` returned `bar`, it attached all the nonlocal variables `bar` needs to run outside of the `foo`'s scope. The closure of a function can be accessed with `.__closure__` attribute:

In [20]:
var.__closure__

(<cell at 0x000001457A457FD0: int object at 0x00007FFFB3762C40>,)

In [21]:
print(type(var.__closure__))

<class 'tuple'>


In [22]:
# Accessing the first element of var's closure
var.__closure__[0]

<cell at 0x000001457A457FD0: int object at 0x00007FFFB3762C40>

In [23]:
# Accessing the contents of each cell in closure
var.__closure__[0].cell_contents

42

Once you access the closure of a function as a tuple, it will contain elements called `cells` with the value of a single nonlocal argument. There can be as many cells inside a closure as the function needs:

In [24]:
outside = 'global variable'  # not in close because it is a global variable
def parent():
    x = 100
    y = 'Hello'
    z = {'name': 'Jon',
         'surname': 'smith'}
    
    def child():
        # as we are printing x, y, and z, they get added to the closure
        print(x, y, z)
        value = 42  # not in the closure because value is not nonlocal
    
    return child

In [25]:
# func is now assigned to `child` function
func = parent()
# print each cell in func's closure
for cell in func.__closure__:
    print(cell.cell_contents, '\n')

100 

Hello 

{'name': 'Jon', 'surname': 'smith'} 



In [26]:
# Print the number of cells in func's closure
len(func.__closure__)

3

In this example, the variables `x, y, z` are nonlocal variables to `child` so they get added to the function's closure. Any other names such as `value` and `outside` are not in the closure because they are not in nonlocal scope. 

Now, consider this trickier example:

In [27]:
var = 'dummy'

def parent(arg):
    
    def child():
        print(arg)
    
    return child

In [28]:
func = parent(var)

We create a `parent` function which takes a single argument and a nested function `child` which prints whatever value passed to `parent`. We call `parent` with `var` ('dummy') and assign the result to `func`. If we call it:

In [29]:
func()

dummy


As expected, it prints out 'dummy'. Now let's delete `var` and call `func` again:

In [30]:
# Delete 'var'
del var
# call func again
func()

dummy


It still prints out 'dummy'. Why? 

You guessed it, it got added to the closure! So, when a value from outer levels of scope gets added to closure, it will stay there unchanged even if we delete the original value!

In [31]:
func.__closure__[0].cell_contents

'dummy'

Even if we did not delete `var` and changed its value, the closure would still contain its old value:

In [37]:
var = 'dummy'

def parent(arg):
    
    def child():
        print(arg)
    
    return child

# Call it as is
my_func = parent(var)
my_func()
# Call after changin var
var = 'new_dummy'
my_func()

dummy
dummy


This concept is going to be important when we talk about decorators in the next section.

Let's go over some of the concepts to make sure you understand:
- Closure is an internal memory of a nested function. It contains all the nonlocal variables stored in a tuple which are essential for the function to run.
- Once a value is stored in a closure, it can be accessed but cannot be overriden if the original value gets deleted or modified
- A nested function is a function defined in another and follows this general pattern:
```python
def parent(arg):
    
    def child():
        print(arg)
    
    return child
```

### Finally, Decorators

> Decorators are functions that modify another function. They can change the functions inputs, its output or even its behavior.

You may have seen decorators when you were creating custom [context managers](https://towardsdatascience.com/how-to-build-custom-context-managers-in-python-31727ffe96e1?source=your_stories_page-------------------------------------) or when you were first introduced to `Flask` (remember `@app.route`?)

In [43]:
def add_one(func):
    
    def wrapper(a):
        return func(a + 1)
    return wrapper

Below, we created a function that squares whatever argument is passed and we are decorating it with `add_one`. `add_one` adds 1 to the argument of the passed function:

In [44]:
@add_one
def square(a):
    return a ** 2

In [45]:
square(5)

36

To use a function as a decorator, just put `@ symbol` followed by the decorating function's name right above the function definition. When we passed 5 to the decorated `square` function, instead of returning 25, it returns 36 because `add_one` takes the argument of `square`, which is 5, and adds one to it and inserts back into our function:

In [46]:
square(10)

121

In this section, we will build `add_one` together. So, let's start with `add_one` that only returns whatever function passed to it:

In [59]:
def square(a):
    return a ** 2

def add_one(func):
    return func

In [60]:
new_square = add_one(square)
new_square(5)

25

For our decorator to return a modified function, it is usually helpful to define a nested function to return:

In [61]:
def add_one(func):
    # Define a new function to modify
    def child(a):
        # Return the result by calling `func` with `a`
        return func(a)
    # Return the nested function
    return child

In [62]:
new_square = add_one(square)
new_square(5)

25

Our decorator is still doing nothing. Inside `add_one` we defined a nested `child` function. `child` only takes one argument and calls whatever function passed to `add_one`. Then, `add_one` returns `child`.

> In this case of nested `child` function, we are assuming `func` passed to `add_one` takes exactly the same number of arguments as `child`.

Now, we can make all the magic happen inside the `child` function. Instead of simply calling the `func`, we want to modify its arguments by adding 1 to them: 

In [63]:
def add_one(func):
    # Define a new function to modify
    def child(a):
        # Add 1 to `a` and then, call
        return func(a + 1)
    # Return the nested function
    return child

Notice `func(a + 1)`? It is calling whatever passed to `add_one` with 1 added to the argument. This time, instead of creating a new variable to store `child`, we will override `square`:

In [64]:
square = add_one(square)
square(5)

36

Now it is returning 36 instead of 25 when we pass 5. How can it use `square` function even when we override it? Good thing we learned closures, because the old `square` is now inside the closure of `child`:

In [65]:
square.__closure__[0].cell_contents

<function __main__.square(a)>

To use `add_one` as a decorator without creating temporary variables, we just add `@add_one` above the `square` function's definition:

In [66]:
@add_one
def square(num):
    return num ** 2

square(7) # returns 64

64